In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

In [2]:
# read file
user_engagement = pd.read_csv('takehome_user_engagement.csv')

In [3]:
# read file
users = pd.read_excel('takehome_users.xlsx')

In [4]:
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:00,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:00,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:00,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:00,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:00,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [5]:
user_engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [6]:
# check missing values
users.isna().sum()

object_id                        0
creation_time                    0
name                             0
email                            0
creation_source                  0
last_session_creation_time    3177
opted_in_to_mailing_list         0
enabled_for_marketing_drip       0
org_id                           0
invited_by_user_id            5583
dtype: int64

In [7]:
# check missing values
user_engagement.isna().sum()

time_stamp    0
user_id       0
visited       0
dtype: int64

In [8]:
users.shape

(12000, 10)

In [9]:
# see how many unique users in "user_engagement"
user_engagement.user_id.nunique()

8823

In [10]:
# see how many unique users in "users"
users.object_id.nunique()

12000

In [11]:
user_engagement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   time_stamp  207917 non-null  object
 1   user_id     207917 non-null  int64 
 2   visited     207917 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [12]:
# convert column to datetime format
user_engagement['time_stamp'] = pd.to_datetime(user_engagement['time_stamp'])

In [13]:
user_engagement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   time_stamp  207917 non-null  datetime64[ns]
 1   user_id     207917 non-null  int64         
 2   visited     207917 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.8 MB


In [14]:
# sort values
user_engagement = user_engagement.sort_values(by=['user_id','time_stamp'])

In [15]:
# check if values sorted correctly
user_engagement.head(30)

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1
5,2013-12-31 03:45:04,2,1
6,2014-01-08 03:45:04,2,1
7,2014-02-03 03:45:04,2,1
8,2014-02-08 03:45:04,2,1
9,2014-02-09 03:45:04,2,1


In [16]:
# get all unique users in user_engagement
total_users = list(user_engagement['user_id'].unique())

In [17]:
# create inital values for users' "adopted" status
adopted = [0.0] * len(total_users)

In [18]:
# create dataframe with user id and adopted status
adopted_user = pd.DataFrame({'adopted': adopted}, index=total_users)

In [19]:
# check if a user has at least three login in any 7-day period
adopted = []
for i in range(user_engagement.shape[0]-2):    # iterate over every row except the last two rows
    user_1 = user_engagement.loc[i, 'user_id']  # the user id in that row
    user_2 = user_engagement.loc[i+2, 'user_id']  # the user id in the 3rd row after
    time_1 = user_engagement.loc[i, 'time_stamp'] # the time in that row
    time_2 = user_engagement.loc[i+2, 'time_stamp']  # the time in the 3rd row after
    user_cond = (user_1 == user_2)    # user ids are same in the initial row and the 3rd row after
    time_cond = ((time_2 - time_1) < timedelta(days=7))  # time difference is within 7 days
    if (user_cond & time_cond): # if the user is the same and the time difference is within 7 days in the 3rd row after
        adopted_user.loc[user_1,'adopted'] = 1  # change the adopted status to 1
    

In [20]:
adopted_user.head()

,adopted
1,0.0
2,1.0
3,0.0
4,0.0
5,0.0


In [21]:
# reset index 
adopted_user = adopted_user.reset_index()

In [22]:
adopted_user.head()

,index,adopted
0,1,0.0
1,2,1.0
2,3,0.0
3,4,0.0
4,5,0.0


In [23]:
# change column name
adopted_user = adopted_user.rename(columns={'index':'user_id'})

In [24]:
adopted_user.head()

,user_id,adopted
0,1,0.0
1,2,1.0
2,3,0.0
3,4,0.0
4,5,0.0


In [25]:
# merge table
users = users.merge(adopted_user, left_on='object_id', right_on='user_id')

In [26]:
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,user_id,adopted
0,1,2014-04-22 03:53:00,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,1,0.0
1,2,2013-11-15 03:45:00,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2,1.0
2,3,2013-03-19 23:14:00,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,3,0.0
3,4,2013-05-21 08:09:00,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,4,0.0
4,5,2013-01-17 10:14:00,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,5,0.0


In [27]:
# see if column "creation_source" has different values in different adopted status
users.groupby('adopted')['creation_source'].value_counts()

adopted  creation_source   
0.0      ORG_INVITE            2635
         SIGNUP                1605
         GUEST_INVITE          1228
         SIGNUP_GOOGLE_AUTH    1153
         PERSONAL_PROJECTS      600
1.0      ORG_INVITE             553
         GUEST_INVITE           360
         SIGNUP                 293
         SIGNUP_GOOGLE_AUTH     232
         PERSONAL_PROJECTS      164
Name: creation_source, dtype: int64

In [28]:
# see if column "opted_in_to_mailing_list" has different values in different adopted status
users.groupby('adopted')['opted_in_to_mailing_list'].value_counts()

adopted  opted_in_to_mailing_list
0.0      0                           5409
         1                           1812
1.0      0                           1188
         1                            414
Name: opted_in_to_mailing_list, dtype: int64

In [29]:
# see if column "enabled_for_marketing_drip" has different values in different adopted status
users.groupby('adopted')['enabled_for_marketing_drip'].value_counts()

adopted  enabled_for_marketing_drip
0.0      0                             6126
         1                             1095
1.0      0                             1356
         1                              246
Name: enabled_for_marketing_drip, dtype: int64

In [30]:
# create predictor features and target variable
X = users[['creation_source', 'opted_in_to_mailing_list', 'enabled_for_marketing_drip']]
y = users['adopted']

In [31]:
# encoding the "creation_source" column
X = pd.get_dummies(X, drop_first=True)

In [32]:
X.head()

,opted_in_to_mailing_list,enabled_for_marketing_drip,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH
0,1,0,0,0,0,0
1,0,0,1,0,0,0
2,0,0,1,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [33]:
# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 21)

In [34]:
# use KNN classifier
knn = KNeighborsClassifier()

In [35]:
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [36]:
y_pred = knn.predict(X_test)

In [37]:
score = knn.score(X_test, y_test)

In [38]:
score

0.8114233907524931

In [39]:
# use random forest classifier
rf_model = RandomForestClassifier()

In [40]:
rf_model.fit(X_train, y_train)

RandomForestClassifier()

In [41]:
y_pred = rf_model.predict(X_test)

In [42]:
score = rf_model.score(X_test, y_test)

In [43]:
score

0.8114233907524931

In [44]:
pd.DataFrame(confusion_matrix(y_test, y_pred))

,0,1
0,1790,0
1,416,0


Not quite sure why the classifier does not predict any adopted values which equals 1. 